## Work
### 請結合前面的知識與程式碼，比較不同的 regularization 的組合對訓練的結果與影響：如 dropout, regularizers, batch-normalization 等

In [1]:
import os
import keras
import itertools
# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.
/home/wan-chu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wan-chu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wan-chu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wan-chu/.local/lib/python3.6/site-packages/tensorflow/py

In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization,Dense,Activation,Dropout

def build_mlp(input_shape, drp_ratio=0.2, output_units=10, num_neurons=[128,128,128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = BatchNormalization()(input_layer)
            x = Dense(units=n_units, name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            x = Dropout(drp_ratio)(x)
            
        else:
            x = Dense(units=n_units, name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            x = Dropout(drp_ratio)(x)
    
    out = Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    
    return model

In [6]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 512
DRP_RATIO=[0.1,0.2,0.4]

In [12]:
results = {}
optimizer_set = [keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=0.95),
                 keras.optimizers.Adam(lr=LEARNING_RATE),
                 keras.optimizers.RMSprop(lr=LEARNING_RATE)]

for i, (opt, drp_ratio) in enumerate(itertools.product(optimizer_set,DRP_RATIO)):
    print('number of %i exp with opt %s and drop ratio %.2f' %(i, str(opt), drp_ratio))
    #keras.backend.clear_session()
    model=build_mlp(input_shape=x_train.shape[1:],drp_ratio=drp_ratio)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    model.fit(x_train,y_train,epochs=EPOCHS,
             validation_data=(x_test,y_test),shuffle=True)
    
    
    exp_name='exp-opt-%s-drp-ratio-%s' %(str(opt),str(drp_ratio))
    
    results[exp_name]=model.history.history
    

number of 0 exp with opt <keras.optimizers.SGD object at 0x7f218b5a5358> and drop ratio 0.10
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 12s 231us/step - loss: 1.8192 - acc: 0.3501 - val_loss: 1.5590 - val_acc: 0.4409
Epoch 2/50
50000/50000 [==============================] - 10s 205us/step - loss: 1.6196 - acc: 0.4205 - val_loss: 1.4658 - val_acc: 0.4751
Epoch 3/50
50000/50000 [==============================] - 10s 193us/step - loss: 1.5458 - acc: 0.4487 - val_loss: 1.4182 - val_acc: 0.4980
Epoch 4/50
50000/50000 [==============================] - 10s 195us/step - loss: 1.5020 - acc: 0.4622 - val_loss: 1.3936 - val_acc: 0.5064
Epoch 5/50
50000/50000 [==============================] - 10s 206us/step - loss: 1.4650 - acc: 0.4737 - val_loss: 1.3739 - val_acc: 0.5098
Epoch 6/50
50000/50000 [==============================] - 10s 207us/step - loss: 1.4265 - acc: 0.4919 - val_loss: 1.3500 - val_acc: 0.5208
Epoch 7/50
50000/50000 

50000/50000 [==============================] - 10s 201us/step - loss: 1.5049 - acc: 0.4626 - val_loss: 1.3832 - val_acc: 0.5097
Epoch 8/50
50000/50000 [==============================] - 10s 203us/step - loss: 1.4857 - acc: 0.4686 - val_loss: 1.3567 - val_acc: 0.5180
Epoch 9/50
50000/50000 [==============================] - 10s 203us/step - loss: 1.4676 - acc: 0.4745 - val_loss: 1.3471 - val_acc: 0.5182
Epoch 10/50
50000/50000 [==============================] - 10s 203us/step - loss: 1.4496 - acc: 0.4818 - val_loss: 1.3458 - val_acc: 0.5174
Epoch 11/50
50000/50000 [==============================] - 10s 203us/step - loss: 1.4293 - acc: 0.4904 - val_loss: 1.3318 - val_acc: 0.5182
Epoch 12/50
50000/50000 [==============================] - 10s 202us/step - loss: 1.4218 - acc: 0.4914 - val_loss: 1.3211 - val_acc: 0.5258
Epoch 13/50
50000/50000 [==============================] - 10s 205us/step - loss: 1.4113 - acc: 0.4923 - val_loss: 1.3236 - val_acc: 0.5239
Epoch 14/50
50000/50000 [=========

50000/50000 [==============================] - 10s 196us/step - loss: 1.5907 - acc: 0.4297 - val_loss: 1.4203 - val_acc: 0.4868
Epoch 15/50
50000/50000 [==============================] - 10s 196us/step - loss: 1.5790 - acc: 0.4312 - val_loss: 1.4173 - val_acc: 0.4925
Epoch 16/50
50000/50000 [==============================] - 10s 196us/step - loss: 1.5728 - acc: 0.4363 - val_loss: 1.4113 - val_acc: 0.4988
Epoch 17/50
50000/50000 [==============================] - 10s 195us/step - loss: 1.5676 - acc: 0.4387 - val_loss: 1.4000 - val_acc: 0.4990
Epoch 18/50
50000/50000 [==============================] - 10s 196us/step - loss: 1.5496 - acc: 0.4463 - val_loss: 1.3948 - val_acc: 0.5003
Epoch 19/50
50000/50000 [==============================] - 10s 198us/step - loss: 1.5486 - acc: 0.4455 - val_loss: 1.3885 - val_acc: 0.5003
Epoch 20/50
50000/50000 [==============================] - 10s 196us/step - loss: 1.5331 - acc: 0.4551 - val_loss: 1.3810 - val_acc: 0.5150
Epoch 21/50
50000/50000 [=======

50000/50000 [==============================] - 12s 247us/step - loss: 1.1367 - acc: 0.5961 - val_loss: 1.2379 - val_acc: 0.5588
Epoch 22/50
50000/50000 [==============================] - 12s 246us/step - loss: 1.1253 - acc: 0.5977 - val_loss: 1.2424 - val_acc: 0.5560
Epoch 23/50
50000/50000 [==============================] - 12s 249us/step - loss: 1.1184 - acc: 0.6024 - val_loss: 1.2407 - val_acc: 0.5538
Epoch 24/50
50000/50000 [==============================] - 12s 248us/step - loss: 1.1026 - acc: 0.6079 - val_loss: 1.2382 - val_acc: 0.5588
Epoch 25/50
50000/50000 [==============================] - 12s 247us/step - loss: 1.1020 - acc: 0.6074 - val_loss: 1.2415 - val_acc: 0.5620
Epoch 26/50
50000/50000 [==============================] - 12s 248us/step - loss: 1.0931 - acc: 0.6104 - val_loss: 1.2545 - val_acc: 0.5566
Epoch 27/50
50000/50000 [==============================] - 12s 247us/step - loss: 1.0843 - acc: 0.6134 - val_loss: 1.2501 - val_acc: 0.5524
Epoch 28/50
50000/50000 [=======

50000/50000 [==============================] - 13s 251us/step - loss: 1.2091 - acc: 0.5690 - val_loss: 1.2369 - val_acc: 0.5548
Epoch 29/50
50000/50000 [==============================] - 13s 251us/step - loss: 1.2020 - acc: 0.5743 - val_loss: 1.2415 - val_acc: 0.5547
Epoch 30/50
50000/50000 [==============================] - 13s 254us/step - loss: 1.2011 - acc: 0.5736 - val_loss: 1.2285 - val_acc: 0.5605
Epoch 31/50
50000/50000 [==============================] - 13s 252us/step - loss: 1.2040 - acc: 0.5705 - val_loss: 1.2352 - val_acc: 0.5549
Epoch 32/50
50000/50000 [==============================] - 13s 252us/step - loss: 1.1888 - acc: 0.5789 - val_loss: 1.2466 - val_acc: 0.5556
Epoch 33/50
50000/50000 [==============================] - 13s 252us/step - loss: 1.1851 - acc: 0.5793 - val_loss: 1.2368 - val_acc: 0.5519
Epoch 34/50
50000/50000 [==============================] - 13s 251us/step - loss: 1.1841 - acc: 0.5814 - val_loss: 1.2370 - val_acc: 0.5564
Epoch 35/50
50000/50000 [=======

50000/50000 [==============================] - 13s 255us/step - loss: 1.4021 - acc: 0.5035 - val_loss: 1.2892 - val_acc: 0.5407
Epoch 36/50
50000/50000 [==============================] - 13s 258us/step - loss: 1.3986 - acc: 0.5059 - val_loss: 1.2907 - val_acc: 0.5356
Epoch 37/50
50000/50000 [==============================] - 13s 253us/step - loss: 1.3900 - acc: 0.5098 - val_loss: 1.2915 - val_acc: 0.5408
Epoch 38/50
50000/50000 [==============================] - 13s 252us/step - loss: 1.3934 - acc: 0.5082 - val_loss: 1.2839 - val_acc: 0.5423
Epoch 39/50
50000/50000 [==============================] - 13s 255us/step - loss: 1.3931 - acc: 0.5066 - val_loss: 1.2893 - val_acc: 0.5382
Epoch 40/50
50000/50000 [==============================] - 13s 254us/step - loss: 1.3839 - acc: 0.5094 - val_loss: 1.2886 - val_acc: 0.5380
Epoch 41/50
50000/50000 [==============================] - 13s 254us/step - loss: 1.3822 - acc: 0.5106 - val_loss: 1.2862 - val_acc: 0.5387
Epoch 42/50
50000/50000 [=======

50000/50000 [==============================] - 12s 240us/step - loss: 1.0421 - acc: 0.6293 - val_loss: 1.2560 - val_acc: 0.5587
Epoch 43/50
50000/50000 [==============================] - 12s 241us/step - loss: 1.0382 - acc: 0.6306 - val_loss: 1.2641 - val_acc: 0.5581
Epoch 44/50
50000/50000 [==============================] - 12s 239us/step - loss: 1.0317 - acc: 0.6344 - val_loss: 1.2761 - val_acc: 0.5579
Epoch 45/50
50000/50000 [==============================] - 12s 240us/step - loss: 1.0258 - acc: 0.6366 - val_loss: 1.2764 - val_acc: 0.5590
Epoch 46/50
50000/50000 [==============================] - 12s 239us/step - loss: 1.0289 - acc: 0.6365 - val_loss: 1.2875 - val_acc: 0.5560
Epoch 47/50
50000/50000 [==============================] - 12s 240us/step - loss: 1.0214 - acc: 0.6364 - val_loss: 1.2852 - val_acc: 0.5556
Epoch 48/50
50000/50000 [==============================] - 12s 241us/step - loss: 1.0147 - acc: 0.6405 - val_loss: 1.2845 - val_acc: 0.5623
Epoch 49/50
50000/50000 [=======

50000/50000 [==============================] - 12s 234us/step - loss: 1.1529 - acc: 0.5945 - val_loss: 1.2569 - val_acc: 0.5563
Epoch 50/50
50000/50000 [==============================] - 12s 235us/step - loss: 1.1439 - acc: 0.5964 - val_loss: 1.2612 - val_acc: 0.5552
number of 8 exp with opt <keras.optimizers.RMSprop object at 0x7f2191d8fb70> and drop ratio 0.40
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 14s 275us/step - loss: 1.9810 - acc: 0.2873 - val_loss: 1.6473 - val_acc: 0.4214
Epoch 2/50
50000/50000 [==============================] - 12s 238us/step - loss: 1.7795 - acc: 0.3612 - val_loss: 1.5771 - val_acc: 0.4472
Epoch 3/50
50000/50000 [==============================] - 12s 237us/step - loss: 1.7169 - acc: 0.3908 - val_loss: 1.5374 - val_acc: 0.4683
Epoch 4/50
50000/50000 [==============================] - 12s 234us/step - loss: 1.6699 - acc: 0.4096 - val_loss: 1.4822 - val_acc: 0.4718
Epoch 5/50
50000/50000 [=====

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""